<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/1_BETO_binario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## finetuning de BETO con el dataset etiquetado original

In [ ]:
!pip install datasets seaborn



In [ ]:
# no lograba bajar el dataset porque
# me fallaba bajar archivos de hugginface y tenia que actualizar esto (JUAN MA me ayudo)
#pip install -U datasets

In [ ]:
from datasets import load_dataset
import pandas as pd
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from collections import Counter

(1) Dataset de etiquetado

https://huggingface.co/datasets/piuba-bigdata/contextualized_hate_speech

Este es el dataset original de etiquetado. Contiene los siguientes campos:

ID : identificador

TITLE: título de la noticia

TEXT: texto del tweet del comentario al Tweet original

CONTEXT: tweet original

HATEFUL: 1 hateful, 0 not hateful

BODY: texto de la noticia

CALLS: si se realizó o no un llamado a la acción

WOMEN: discriminación hacia mujeres

LGBTI: discriminación hacia grupos LGBTI

RACISM: discriminación asociada a racismo

CLASS: discriminación vinculada a pobreza, villas

POLITICS: odio o violencia vinculada a las ideas políticas

DISABLED: discriminación a las personas con discapacidad

APPEARENCE: discriminación en base al aspecto físico (incluye especialmente la gordofobia)

CRIMINAL: discriminación hacia personas en el sistema penal o similares



In [ ]:
# para ver las versiones de las librerias
#pip freeze

In [ ]:
from datasets import load_dataset

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

In [ ]:
##Esto era para levantarlo desde parquet cuando no pude hacerlo con HUGGINFACE
## cargo  train, test y dev por separado
##ds1_train=pd.read_parquet('train/train-00000-of-00001.parquet', engine='pyarrow')
###ds1_train

El dataset 1 contiene 56869 tw etiquetados.
El dataset se dividió en 36420 de train, test:11343 y vaLidacion:9106

In [ ]:
ds1_train=ds1["train"]
ds1_train

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 36420
})

In [ ]:
example1= ds1_train[30355]
example1

{'id': 374451,
 'title': 'Asalto y tiroteo en un colectivo en Villa Lugano: murió un ladrón de 15 años',
 'text': '@usuario Uno menos.',
 'context_tweet': 'Asalto y tiroteo en un colectivo en Villa Lugano: murió un ladrón de 15 años https://t.co/arXOD0w5xp',
 'HATEFUL': 1,
 'body': 'Dos delincuentes subieron a un colectivo de la línea 47 en el barrio porteño de Villa Lugano, muy cerca de la General Paz y autopista Riccheri, con intenciones de asaltar a todos pero se encontraron con que entre los pasajeros viajaba una mujer policía vestida de civil que resistió el robo a los tiros.\n\nSegún fuentes oficiales, el hecho ocurrió durante la madrugada en el interno 106 de la línea 47. La mujer policía que estaba franco de servicio defendió a los pasajeros y al chofer y se enfrentó a los delincuentes.\n\nTodo comenzó cuando en la parada ubicada en Albariño y Riestra, subieron dos personas armadas. Los testigos comentaron que mientras uno le robaba el celular a una mujer y el otro amenazaba co

## Para el entrenamiento se usaron los siguientes tutoriales y documentos:

Uso  de base el siguiente tutorial para hacer el finetuning:
https://huggingface.co/blog/sentiment-analysis-python
y este:
https://huggingface.co/docs/transformers/training


La idea es utilizar la librería tranbsformers de huggingface para hacerle finetuning a un BETO original, es decir la version de BERT pre-entrenada con español de GOOGLE pero que no haya sido reentrenado con ningún otro dataset.
Aqui para usar transformers de hugginface
https://github.com/huggingface/transformers?tab=readme-ov-file



In [ ]:
import torch
torch.cuda.is_available()

True

### 1. pre-process data

In [ ]:
 ds1['train'][0]

{'id': 343726,
 'title': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad',
 'text': '@usuario Uno menos',
 'context_tweet': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad https://t.co/UrJGRAdEvD',
 'HATEFUL': 1,
 'body': 'Un hombre de 46 años, que cumplía una condena en prisión y fue puesto en libertad por el coronavirus, murió el mismo día que dejó la cárcel baleado por un primo, durante los "festejos" por dejar la prisión.\n\nSari Salem Wardat, de 46 años, recibió un disparo desde atrás en un aparente accidente mientras caminaba hacia miembros de la familia y vecinos que se habían reunido para darle la bienvenida a su casa, en Jordania.\n\nUn video que circuló en las redes sociales \u200bmuestra a Sari llegando a su casa en un auto azul rodeado de tiros de "celebración". El primo se acerca de inmediato a Sari, que sostiene una pistola y la apunta hacia el cielo.\n\nEl detenido

In [ ]:
ds_train_small= ds1['train'].select_columns(["text","HATEFUL"])
ds_test_small= ds1['test'].select_columns(["text","HATEFUL"])

In [ ]:
##aca les coloco el mismo nombre que espera el modelo (me parece cualquiera pero no logre que ande sino)
ds_train_small=ds_train_small.rename_column("HATEFUL", "label")
ds_test_small=ds_test_small.rename_column("HATEFUL", "label")

In [ ]:
ds_train_small

Dataset({
    features: ['text', 'label'],
    num_rows: 36420
})

La versión BETO que se utiliza es la siguiente:


https://github.com/dccuchile/beto?tab=readme-ov-file

Nombre del modelo: dccuchile/bert-base-spanish-wwm-uncased


In [ ]:
import torch
from transformers import BertForMaskedLM, BertTokenizer

Utiizo las librerías de transformers para preparar los daos de entrenamiento

Lo primero es tokenizar en función del modelo seleccionado

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

Le aplico la función tokenizer del modelo BETO a mis dos conjuntos, el de entrenamiento y el de test.

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = ds_train_small.map(preprocess_function, batched=True)
tokenized_test = ds_test_small.map(preprocess_function, batched=True)

In [ ]:
tokenized_train[0]

{'text': '@usuario Uno menos',
 'label': 1,
 'input_ids': [4, 985, 8563, 1614, 1885, 5],
 'token_type_ids': [0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

La siguiente función (según el ejmplo) transforma los datos de entrenamiento agregandole el padding

Se usará luego en el Trainer

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 2 Entreno el modelo.

Voy a entrenar el modelo con el TRAINER de transformers. Instancio el modelo pre-entrenado BETO y aca selecciono 2 labels

dccuchile/bert-base-spanish-wwm-uncased


AGUS : ACA TENDRIA QUE CAMBIAR LA CANTIDAD DE LABELS PARA PODER ENTRENAR CON TODAS LAS ETIQUETAS?

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
##FUNCION PARA COMPUTAR LAS METRIcas

import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#! pip install -U accelerate
! pip install -U transformers

Revision de parametros de TRAINER
https://huggingface.co/docs/transformers/main_classes/trainer

Definicipon de parametros
https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/trainer#transformers.TrainingArguments



In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "beto-hate_speech_ar_bin2"

training_args = TrainingArguments(
	 output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
# Train pre-trained model
trainer.train()

Step,Training Loss
500,0.357700
1000,0.305800
1500,0.297100
2000,0.290300
2500,0.260600
3000,0.214800
3500,0.209200
4000,0.213500
4500,0.222000


TrainOutput(global_step=4554, training_loss=0.2629721886136839, metrics={'train_runtime': 928.246, 'train_samples_per_second': 78.471, 'train_steps_per_second': 4.906, 'total_flos': 2173085595096240.0, 'train_loss': 0.2629721886136839, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

<ipython-input-21-6bc43949e8a2>:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for f1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/f1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


{'eval_loss': 0.3138335943222046,
 'eval_accuracy': 0.8942960416115666,
 'eval_f1': 0.6361153262518968,
 'eval_runtime': 66.1171,
 'eval_samples_per_second': 171.559,
 'eval_steps_per_second': 10.723,
 'epoch': 2.0}

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


Voy a probar el modelo entrenado y guardado

In [ ]:
from transformers import pipeline
data = ["hay que matarlos a todos", "Vamos a la plaza a marchar"]
specific_model = pipeline(model="natdebandi/beto-hate_speech_ar_bin")
specific_model(data)

[{'label': 'LABEL_1', 'score': 0.951015830039978},
 {'label': 'LABEL_0', 'score': 0.9810741543769836}]

In [ ]:
data = ["hay que matarlos a todos"]


pipe("sos un hijo de puta")

[{'label': 'LABEL_0', 'score': 0.967201828956604}]